In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#from langchain.chat_models import init_chat_model
#model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")


In [3]:

#print(os.getenv("GROQ_API_KEY"))


In [4]:
#!pip install -U langchain-huggingface sentence-transformers


In [5]:
from langsmith import traceable
from langchain_groq import ChatGroq
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "groq"
MODEL_NAME = "llama-3.3-70b-versatile" 
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

# Initialize Groq client
llm = ChatGroq(
    model=MODEL_NAME,
    temperature=0.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
    ]
    return call_groq(messages)

@traceable(run_type="llm")
def call_groq(messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0) -> str:
    return llm.invoke(messages)

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content


C:\Users\Aniket\Documents\Clg\LLM - part 2\myllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


question = "What is LangSmith used for?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"website": "www.google.com"}})
print(ai_answer)

In [7]:
def ask_question(question: str):
    return langsmith_rag(question, langsmith_extra={"metadata": {"website": "www.google.com"}})

In [8]:
print(ask_question("how many odi centuries does virat kohli have ? "))

I don't know the current number of ODI centuries Virat Kohli has. The provided context does not contain information about Virat Kohli's cricket statistics.




here we can see it does not work when asked an external question , and only answers the questions which can be answers based on the langsmith document. 


In [9]:
print(ask_question("how is langsmith better than other frameworks ?"))

LangSmith is better than other frameworks because it is framework agnostic, allowing it to be used with or without LangChain's open source frameworks. This flexibility, combined with its ability to monitor and evaluate applications, enables developers to ship quickly and with confidence. Additionally, LangSmith's simple data format makes it easy to export and import trace data.
